# WW2 Fleet Battle — mini “World of Warships” web game
This notebook generates a single-page web game (`index.html`) in the same folder and opens it in Google Chrome.

**Controls**
- `1` `2` `3` `4`: select ship (3 destroyers + 1 battleship)
- `W/S`: throttle forward/back (high inertia; ships accelerate slowly)
- `A/D`: turn left/right
- Aim with your mouse (turret follows cursor)
- Left click: fire shells at the clicked point (impact-only)
- `Q`: fire torpedo (if off cooldown)
- `E`: smoke screen (countermeasure)
- `F`: fire extinguisher (stop onboard fire)
- `R`: restart level

**Goal**
Sink the enemy fleet to advance levels (AI gets tougher).

In [84]:
from __future__ import annotations
from pathlib import Path
import os
import re

BASE = Path(r"C:\Users\fabla\OneDrive - Universidad Politécnica de Cartagena\freetime\AFK")
out_path = BASE / "index.html"

def replace_once(text: str, pattern: str, repl, *, flags: int = 0, label: str = "") -> str:
    new_text, n = re.subn(pattern, repl, text, count=1, flags=flags)
    if n != 1:
        raise RuntimeError(f"Expected 1 match for {label or pattern!r}, got {n}")
    return new_text

def replace_maybe(text: str, pattern: str, repl, *, flags: int = 0) -> tuple[str, int]:
    return re.subn(pattern, repl, text, flags=flags)

html = out_path.read_text(encoding="utf-8")

# 1) CV can also launch recon (Shift+T), BB stays same
if "if (e.shiftKey) tryRecon(s);" not in html:
    html = replace_once(
        html,
        r"if \(e\.code === 'KeyT'\) \{\s*const s = state\.player\[state\.selected\];\s*if \(!s\) return;\s*if \(s\.kind\.type === 'BB'\) tryRecon\(s\);\s*else if \(s\.kind\.type === 'CV'\) tryLaunchFighters\(s\);\s*\}",
        "if (e.code === 'KeyT') {\n"
        "      const s = state.player[state.selected];\n"
        "      if (!s) return;\n"
        "      if (s.kind.type === 'BB') tryRecon(s);\n"
        "      else if (s.kind.type === 'CV') {\n"
        "        if (e.shiftKey) tryRecon(s);\n"
        "        else tryLaunchFighters(s);\n"
        "      }\n"
        "    }",
        flags=re.MULTILINE,
        label="KeyT handler"
    )

# 2) Allow recon from CV too
if "ship.kind.type !== 'CV'" not in html:
    html = replace_once(
        html,
        r"if \(ship\.kind\.type !== 'BB'\) \{ if \(!silent && ship\.team === 'P'\) showMsg\('Recon plane only available on Battleships'\); return; \}",
        "if (ship.kind.type !== 'BB' && ship.kind.type !== 'CV') { if (!silent && ship.team === 'P') showMsg('Recon plane only available on Battleships/Carriers'); return; }",
        label="tryRecon ship kind gate"
    )

# 3) Islands controlled by US: only spawn US (P) coastal batteries
html = replace_maybe(
    html,
    r"placeTeam\('P'\);\s*placeTeam\('E'\);",
    "placeTeam('P');",
)[0]

if "const maxPerTeam = 7;" not in html:
    html = replace_once(
        html,
        r"const maxPerTeam = 6;",
        "// Islands are controlled by the US: only spawn US (P) batteries\n    const maxPerTeam = 7;",
        label="spawnCoastalBatteries maxPerTeam"
    )
html = replace_maybe(
    html,
    r"tag: 'Coastal Battery'",
    "tag: 'US Coastal Battery'",
 )[0]

# 4) Bots avoid hostile battery zones (enemy avoids US batteries)
INSERT_AVOID = (
    "function avoidHostileBatteries(ship, tx, ty) {\n"
    "    if (!ship || !ship.alive) return { x: tx, y: ty };\n"
    "    const hostile = state.batteries.filter(b => b.alive && b.team !== ship.team);\n"
    "    if (!hostile.length) return { x: tx, y: ty };\n"
    "    let x = tx, y = ty;\n"
    "    const danger = BATTERY.gunRange * 0.92;\n"
    "    for (const b of hostile) {\n"
    "      const dx = x - b.x, dy = y - b.y;\n"
    "      const d = Math.hypot(dx, dy);\n"
    "      if (d < danger) {\n"
    "        const a = (d > 1e-3) ? Math.atan2(dy, dx) : rand(0, TAU);\n"
    "        x = b.x + Math.cos(a) * danger;\n"
    "        y = b.y + Math.sin(a) * danger;\n"
    "      }\n"
    "    }\n"
    "    return { x: clamp(x, 0, WORLD.w), y: clamp(y, 0, WORLD.h) };\n"
    "}\n\n"
 )
if "function avoidHostileBatteries(" not in html:
    html = replace_once(
        html,
        r"function pickTargetDetectable\(fromShip, candidates\) \{[\s\S]*?\}\n\n\s*function aiStep\(",
        lambda m: m.group(0).replace("function aiStep(", INSERT_AVOID + "function aiStep("),
        flags=re.MULTILINE,
        label="insert avoidHostileBatteries"
    )

# Formation follow target avoidance
html = replace_maybe(
    html,
    r"const fp = formationPoint\(leader, ship\);\s*const dForm = steerToPoint\(ship, clamp\(fp\.x, 0, WORLD\.w\), clamp\(fp\.y, 0, WORLD\.h\), dt, ship\.kind\.maxSpeed\);",
    "const fp = formationPoint(leader, ship);\n"
    "      let tgt = { x: clamp(fp.x, 0, WORLD.w), y: clamp(fp.y, 0, WORLD.h) };\n"
    "      tgt = avoidHostileBatteries(ship, tgt.x, tgt.y);\n"
    "      const dForm = steerToPoint(ship, tgt.x, tgt.y, dt, ship.kind.maxSpeed);",
 )[0]

# Patrol avoidance: adjust waypoint before steering
if "avoidHostileBatteries(ship, ship.ai.wpX, ship.ai.wpY)" not in html:
    html = replace_once(
        html,
        r"if \(!best\) \{[\s\S]*?ship\.ai\.wpUntil = state\.time \+ rand\(4\.0, 7\.5\);\s*\}\s*const desired = angleTo\(ship\.x, ship\.y, ship\.ai\.wpX, ship\.ai\.wpY\);",
        lambda m: m.group(0).replace(
            "const desired = angleTo(ship.x, ship.y, ship.ai.wpX, ship.ai.wpY);",
            "const adj = avoidHostileBatteries(ship, ship.ai.wpX, ship.ai.wpY);\n"
            "      ship.ai.wpX = adj.x; ship.ai.wpY = adj.y;\n"
            "      const desired = angleTo(ship.x, ship.y, ship.ai.wpX, ship.ai.wpY);",
        ),
        flags=re.MULTILINE,
        label="aiStep patrol avoidance"
    )

# Orbit avoidance: adjust tx/ty before final desired angle
if "const adj = avoidHostileBatteries(ship, tx, ty);" not in html:
    html = replace_once(
        html,
        r"let tx = best\.x, ty = best\.y;[\s\S]*?\}\s*const desired = angleTo\(ship\.x, ship\.y, tx, ty\);",
        lambda m: m.group(0).replace(
            "const desired = angleTo(ship.x, ship.y, tx, ty);",
            "const adj = avoidHostileBatteries(ship, tx, ty);\n"
            "    tx = adj.x; ty = adj.y;\n"
            "    const desired = angleTo(ship.x, ship.y, tx, ty);",
        ),
        flags=re.MULTILINE,
        label="aiStep orbit avoidance"
    )

# 5) Minimap: make grey zone obvious (land + battery danger rings)
if "// Grey zone: land + coastal battery danger areas" not in html:
    html = replace_once(
        html,
        r"const toR = \(x, y\) => \(\{ x: rx \+ \(x / WORLD\.w\) \* rw, y: ry \+ \(y / WORLD\.h\) \* rh \}\);",
        "const toR = (x, y) => ({ x: rx + (x / WORLD.w) * rw, y: ry + (y / WORLD.h) * rh });\n"
        "    // Grey zone: land + coastal battery danger areas\n"
        "    const m = mapById(state.mapId);\n"
        "    // Land silhouettes\n"
        "    ctx.save();\n"
        "    ctx.globalAlpha = 0.36;\n"
        "    ctx.fillStyle = 'rgba(170,170,170,0.45)';\n"
        "    for (const c of m.land) {\n"
        "      const q = toR(c.x, c.y);\n"
        "      const rs = ((rw + rh) * 0.5) / ((WORLD.w + WORLD.h) * 0.5);\n"
        "      const r = Math.max(1.2, c.r * rs);\n"
        "      ctx.beginPath(); ctx.arc(q.x, q.y, r, 0, TAU); ctx.fill();\n"
        "    }\n"
        "    ctx.restore();\n"
        "    // Battery danger circles (US-controlled islands)\n"
        "    ctx.save();\n"
        "    ctx.globalAlpha = 0.22;\n"
        "    ctx.fillStyle = 'rgba(190,190,190,0.22)';\n"
        "    ctx.strokeStyle = 'rgba(220,220,220,0.28)';\n"
        "    ctx.lineWidth = 1;\n"
        "    for (const b of state.batteries) {\n"
        "      if (!b.alive) continue;\n"
        "      if (b.team !== 'P') continue;\n"
        "      const q = toR(b.x, b.y);\n"
        "      const rs = ((rw + rh) * 0.5) / ((WORLD.w + WORLD.h) * 0.5);\n"
        "      const r = Math.max(1.2, b.kind.gunRange * rs);\n"
        "      ctx.beginPath(); ctx.arc(q.x, q.y, r, 0, TAU); ctx.fill();\n"
        "      ctx.beginPath(); ctx.arc(q.x, q.y, r, 0, TAU); ctx.stroke();\n"
        "    }\n"
        "    ctx.restore();",
        flags=re.MULTILINE,
        label="drawRadar overlay"
    )

out_path.write_text(html, encoding="utf-8")
print(f"Wrote {out_path}")
print("- Shift+T on CV launches recon; T launches fighters")
print("- US-only island batteries; bots avoid hostile battery zones")
print("- Minimap shows land + battery grey-zone rings")

# Convenience: open in browser
try:
    os.startfile(out_path)
except Exception as e:
    print("Could not auto-open:", e)

Wrote C:\Users\fabla\OneDrive - Universidad Politécnica de Cartagena\freetime\AFK\index.html
- Shift+T on CV launches recon; T launches fighters
- US-only island batteries; bots avoid hostile battery zones
- Minimap shows land + battery grey-zone rings
